# Predicción de Cantidades Vendidas por Grupo (Prophet)

Este notebook ejecuta un flujo de predicción de cantidades vendidas usando Prophet, generando un modelo y predicción por cada grupo de `REGION_CATEGORIA_PRODUCTO`. Se exportan todas las predicciones y métricas a Snowflake, y se visualiza solo el 10% de los grupos seleccionados al azar.

## 1. Importar Librerías y Configuración

Importamos las librerías necesarias y configuramos logging y warnings.

In [1]:
import pandas as pd
import numpy as np
import snowflake.connector
import logging
from datetime import datetime
import os
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.subplots as sp
from tqdm import tqdm
import random
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

# Suppress Prophet and related loggers
for logger_name in ['prophet', 'fbprophet', 'cmdstanpy']:
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)

c:\Users\joey_\AppData\Local\Programs\Python\Python310\lib\site-packages\snowflake\connector\options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(
c:\Users\joey_\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\joey_\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Carga de Datos desde Snowflake

Definimos funciones para conectar y cargar los dataframes históricos y de escenarios desde Snowflake, sin filtrar por REGION_CATEGORIA_PRODUCTO.

In [ ]:
def get_snowflake_connection():
    try:
        conn = snowflake.connector.connect(
            user='XXXXXXXXXXXXXXX',
            password='XXXXXXXXXXXXXXX',
            account='XXXXXXXXXXXXXXX',
            warehouse='XXXXXXXXXXXXX',
            database='XXXXXXXXXXXXXXXXXXXXX',
            schema='XXXXXXXXXXXXXX'
        )
        logging.info("Conexión a Snowflake exitosa")
        return conn
    except Exception as e:
        logging.error(f"Error de conexión a Snowflake: {e}")
        raise

def load_snowflake_views():
    required_cols = ['MES', 'REGION_CATEGORIA_PRODUCTO',  'AVG_PRECIO_LISTA', 'AVG_DESC_PORCENTAJE']
    conn = get_snowflake_connection()
    try:
        df_hist = pd.read_sql("SELECT * FROM VW_VENTAS_HISTORICO_M3", conn)
        df_fcst1 = pd.read_sql("SELECT * FROM VW_VENTAS_FCST_FEATURES_1", conn)
        df_fcst2 = pd.read_sql("SELECT * FROM VW_VENTAS_FCST_FEATURES_2", conn)
        
        # Validate columns
        for df, name in [(df_hist, 'Histórico'), (df_fcst1, 'Escenario 1'), (df_fcst2, 'Escenario 2')]:
            missing_cols = [col for col in required_cols if col not in df.columns]
            if missing_cols:
                raise ValueError(f"Missing columns in {name}: {missing_cols}")
        
        logging.info(f"Histórico: {len(df_hist)} registros, Escenario 1: {len(df_fcst1)}, Escenario 2: {len(df_fcst2)}")
        return df_hist, df_fcst1, df_fcst2
    finally:
        conn.close()

# Load data
df_hist, df_fcst1, df_fcst2 = load_snowflake_views()
print(f"Histórico: {df_hist.shape}, Escenario 1: {df_fcst1.shape}, Escenario 2: {df_fcst2.shape}")

2025-06-29 00:08:34,010 - INFO - Snowflake Connector for Python Version: 3.15.0, Python Version: 3.10.11, Platform: Windows-10-10.0.19045-SP0
2025-06-29 00:08:34,012 - INFO - Connecting to GLOBAL Snowflake domain
2025-06-29 00:08:34,012 - INFO - Connecting to GLOBAL Snowflake domain
2025-06-29 00:08:35,813 - INFO - Conexión a Snowflake exitosa
2025-06-29 00:08:35,813 - INFO - Conexión a Snowflake exitosa
2025-06-29 00:08:41,702 - INFO - Histórico: 3984 registros, Escenario 1: 1680, Escenario 2: 1680
2025-06-29 00:08:41,702 - INFO - Histórico: 3984 registros, Escenario 1: 1680, Escenario 2: 1680


Histórico: (3984, 6), Escenario 1: (1680, 5), Escenario 2: (1680, 5)


## 3. Preparación de Datos para Prophet por Grupo

Identificamos todos los valores únicos de `REGION_CATEGORIA_PRODUCTO`. Para cada grupo, preparamos los dataframes históricos y de escenarios, escalando variables y agregando columnas requeridas para Prophet.

In [3]:
# Exogenous variables
exog_vars = ['AVG_PRECIO_LISTA', 'AVG_DESC_PORCENTAJE']

# Identify unique groups
grupos = df_hist['REGION_CATEGORIA_PRODUCTO'].unique()
print(f"Total de grupos: {len(grupos)}")

# Initialize result and metrics lists
resultados_todos = []
metricas_todos = []

Total de grupos: 140


## 4. Entrenamiento y Predicción por Grupo

Iteramos sobre cada grupo. Para cada uno, entrenamos un modelo Prophet con variables exógenas, generamos predicciones para ambos escenarios, desescalamos resultados y calculamos intervalos de confianza.

In [4]:
from tqdm import tqdm

# Clear lists to avoid duplicates
resultados_todos = []
metricas_todos = []

for grupo in tqdm(grupos, desc="Procesando grupos Prophet"):
    # Filter data by group
    hist_g = df_hist[df_hist['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
    fcst1_g = df_fcst1[df_fcst1['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
    fcst2_g = df_fcst2[df_fcst2['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
    
    # Skip groups with insufficient data
    if len(hist_g) < 12 or len(fcst1_g) == 0 or len(fcst2_g) == 0:
        logging.warning(f"Grupo {grupo} omitido: pocos datos (Hist: {len(hist_g)}, Fcst1: {len(fcst1_g)}, Fcst2: {len(fcst2_g)})")
        continue
    
    # Prepare columns
    hist_g['ds'] = pd.to_datetime(hist_g['MES'])
    hist_g['y'] = hist_g['M3_VENDIDOS']
    fcst1_g['ds'] = pd.to_datetime(fcst1_g['MES'])
    fcst2_g['ds'] = pd.to_datetime(fcst2_g['MES'])
    
    # Scale variables
    target_scaler = StandardScaler()
    exog_scaler = StandardScaler()
    hist_g['y_scaled'] = target_scaler.fit_transform(hist_g[['y']]).flatten()
    scaled_exog = exog_scaler.fit_transform(hist_g[exog_vars])
    for idx, var in enumerate(exog_vars):
        hist_g[f'{var}_scaled'] = scaled_exog[:, idx]
    
    def scale_exog(df):
        scaled = exog_scaler.transform(df[exog_vars])
        for idx, var in enumerate(exog_vars):
            df[f'{var}_scaled'] = scaled[:, idx]
        return df
    
    fcst1_g = scale_exog(fcst1_g)
    fcst2_g = scale_exog(fcst2_g)
    exog_vars_scaled = [f'{var}_scaled' for var in exog_vars]
    
    # Train Prophet model
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    for var in exog_vars_scaled:
        m.add_regressor(var)
    m.fit(hist_g[['ds', 'y_scaled'] + exog_vars_scaled].rename(columns={'y_scaled': 'y'}))
    
    # Prepare future dataframes
    def prepare_future_scaled(df_fcst):
        return df_fcst[['ds'] + exog_vars_scaled].copy()
    
    future1 = prepare_future_scaled(fcst1_g)
    future2 = prepare_future_scaled(fcst2_g)
    
    # Generate predictions
    forecast1 = m.predict(future1)
    forecast2 = m.predict(future2)
    
    # Inverse scale predictions
    fcst1_g['CANTIDAD_VENDIDA_PRED'] = target_scaler.inverse_transform(forecast1[['yhat']]).flatten()
    fcst1_g['IC_95_LOWER'] = target_scaler.inverse_transform(forecast1[['yhat_lower']]).flatten()
    fcst1_g['IC_95_UPPER'] = target_scaler.inverse_transform(forecast1[['yhat_upper']]).flatten()
    fcst2_g['CANTIDAD_VENDIDA_PRED'] = target_scaler.inverse_transform(forecast2[['yhat']]).flatten()
    fcst2_g['IC_95_LOWER'] = target_scaler.inverse_transform(forecast2[['yhat_lower']]).flatten()
    fcst2_g['IC_95_UPPER'] = target_scaler.inverse_transform(forecast2[['yhat_upper']]).flatten()
    
    # Add metadata
    hist_g['TIPO'] = 'HISTÓRICO'
    hist_g['IC_95_LOWER'] = np.nan
    hist_g['IC_95_UPPER'] = np.nan
    hist_g['DIF_PCT'] = hist_g['y'].pct_change() * 100
    fcst1_g['TIPO'] = 'ESCENARIO 1'
    fcst2_g['TIPO'] = 'ESCENARIO 2'
    
    # Calculate percentage differences
    last_hist = hist_g['y'].iloc[-1]
    def calc_diff(df, col, first_ref=None):
        diffs = df[col].pct_change() * 100
        if first_ref is not None and len(diffs) > 0:
            diffs.iloc[0] = 100 * (df[col].iloc[0] - first_ref) / ((abs(df[col].iloc[0]) + abs(first_ref)) / 2)
        return diffs
    
    fcst1_g['DIF_PCT'] = calc_diff(fcst1_g, 'CANTIDAD_VENDIDA_PRED', first_ref=last_hist)
    fcst2_g['DIF_PCT'] = calc_diff(fcst2_g, 'CANTIDAD_VENDIDA_PRED', first_ref=last_hist)
    
    # Consolidate results
    hist_tabla = hist_g[['ds', 'REGION_CATEGORIA_PRODUCTO', 'y', 'TIPO', 'IC_95_LOWER', 'IC_95_UPPER', 'DIF_PCT']].rename(columns={'y': 'M3_PRODUCTO'})
    sc1 = fcst1_g[['ds', 'REGION_CATEGORIA_PRODUCTO', 'CANTIDAD_VENDIDA_PRED', 'TIPO', 'IC_95_LOWER', 'IC_95_UPPER', 'DIF_PCT']].rename(columns={'CANTIDAD_VENDIDA_PRED': 'M3_PRODUCTO'})
    sc2 = fcst2_g[['ds', 'REGION_CATEGORIA_PRODUCTO', 'CANTIDAD_VENDIDA_PRED', 'TIPO', 'IC_95_LOWER', 'IC_95_UPPER', 'DIF_PCT']].rename(columns={'CANTIDAD_VENDIDA_PRED': 'M3_PRODUCTO'})
    
    # Combine and append results
    resultados_todos.append(pd.concat([hist_tabla, sc1, sc2], ignore_index=True))
    
    def calcular_metricas(hist_g, m, target_scaler, exog_vars_scaled, grupo, metricas_todos, exog_vars, exog_scaler):
        def smape(y_true, y_pred):
            denominator = np.abs(y_true) + np.abs(y_pred)
            # Use np.where instead of .where for numpy arrays
            denominator_safe = np.where(denominator != 0, denominator, 1)
            return 100 * np.mean(2 * np.abs(y_pred - y_true) / denominator_safe)
        
        # Use the last 7 rows or all if fewer than 7
        n_test = min(7, len(hist_g))
        test_hist = hist_g.tail(n_test).copy()
        
        # Prepare future dataframe for prediction
        future_test = test_hist[['ds'] + exog_vars_scaled]
        forecast_test = m.predict(future_test)
        
        # Inverse scale predictions
        y_true = target_scaler.inverse_transform(test_hist[['y_scaled']]).flatten()
        y_pred = target_scaler.inverse_transform(forecast_test[['yhat']]).flatten()
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        smape_val = smape(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        
        # Determine model quality
        if r2 >= 0.8 and smape_val < 20:
            quality = 'BUENO'
        elif r2 >= 0.5 or smape_val < 30:
            quality = 'ACEPTABLE'
        else:
            quality = 'POBRE'
        
        # Append metrics
        metricas_todos.append({
            'REGION_CATEGORIA_PRODUCTO': grupo,
            'RMSE': rmse,
            'MAE': mae,
            'SMAPE': smape_val,
            'R2': r2,
            'QUALITY': quality
        })
        
    # Calculate metrics
    calcular_metricas(hist_g, m, target_scaler, exog_vars_scaled, grupo, metricas_todos, exog_vars, exog_scaler)

Procesando grupos Prophet:   0%|          | 0/140 [00:00<?, ?it/s]2025-06-29 00:08:42,950 - DEBUG - cmd: where.exe tbb.dll
cwd: None
2025-06-29 00:08:42,950 - DEBUG - cmd: where.exe tbb.dll
cwd: None
2025-06-29 00:08:43,263 - DEBUG - Adding TBB (c:\Users\joey_\AppData\Local\Programs\Python\Python310\lib\site-packages\prophet\stan_model\cmdstan-2.33.1\stan\lib\stan_math\lib\tbb) to PATH
2025-06-29 00:08:43,263 - DEBUG - Adding TBB (c:\Users\joey_\AppData\Local\Programs\Python\Python310\lib\site-packages\prophet\stan_model\cmdstan-2.33.1\stan\lib\stan_math\lib\tbb) to PATH
2025-06-29 00:08:43,344 - DEBUG - input tempfile: C:\Users\joey_\AppData\Local\Temp\tmpaz7qzaug\3vlx9l2b.json
2025-06-29 00:08:43,344 - DEBUG - input tempfile: C:\Users\joey_\AppData\Local\Temp\tmpaz7qzaug\3vlx9l2b.json
2025-06-29 00:08:43,371 - DEBUG - input tempfile: C:\Users\joey_\AppData\Local\Temp\tmpaz7qzaug\molq7223.json
2025-06-29 00:08:43,383 - DEBUG - idx 0
2025-06-29 00:08:43,371 - DEBUG - input tempfile: C:

In [5]:
print(f"Grupos procesados: {len(resultados_todos)}")
if len(resultados_todos) != len(grupos):
    logging.warning(f"Se esperaban {len(grupos)} grupos, pero se procesaron {len(resultados_todos)}")

Grupos procesados: 140


## 5. Cálculo de Métricas por Grupo

Para cada grupo, calculamos métricas de desempeño (RMSE, MAE, SMAPE, R2) en el histórico reciente y determinamos la calidad del modelo.

In [6]:

df_metricas = pd.DataFrame(metricas_todos)
print(f"Total de grupos con métricas: {len(df_metricas)}")
if len(df_metricas) != len(grupos):
    logging.warning(f"Se esperaban {len(grupos)} métricas, pero se generaron {len(df_metricas)}")

# Display metrics
df_metricas
df_metricas.describe()

Total de grupos con métricas: 140


,REGION_CATEGORIA_PRODUCTO,RMSE,MAE,SMAPE,R2,QUALITY
0,Antioquia-Bebida de Té-Té Negro 500mL x 6uds,9.020197e-02,7.577871e-02,4.436687,0.796148,ACEPTABLE
1,Antioquia-Gaseosa-Uva 600mL x 24uds,6.997149e-01,5.059958e-01,8.171640,0.645291,ACEPTABLE
2,Antioquia-Jugo-Jugo Naranja 2000mL x 24uds,1.595094e+00,1.298119e+00,9.428326,0.779577,ACEPTABLE
3,Antioquia-Agua-Agua Sin Gas 1L x 1uds,3.301917e-02,2.170066e-02,4.809865,0.842816,BUENO
4,Antioquia-Jugo-Jugo Lima 2000mL x 12uds,1.014950e+00,6.489529e-01,10.578001,-0.164793,ACEPTABLE
...,...,...,...,...,...,...
135,Valle del Cauca-Gaseosa-Uva 600mL x 24uds,2.686113e-01,2.040589e-01,4.196201,0.861654,BUENO
136,Valle del Cauca-Bebida de Té-Té Verde 500mL x ...,2.047673e-01,1.621055e-01,6.572519,0.556165,ACEPTABLE
137,Boyacá-Agua-Agua Sin Gas 1L x 1uds,3.573191e-07,2.994513e-07,0.001083,1.000000,BUENO
138,Boyacá-Bebida de Té-Té Verde 300mL x 6uds,5.569099e-07,4.525358e-07,0.000839,1.000000,BUENO


,RMSE,MAE,SMAPE,R2
count,1.400000e+02,1.400000e+02,140.000000,140.000000
mean,3.108364e-01,2.457713e-01,13.923141,0.656317
std,4.516201e-01,3.525595e-01,11.004740,0.410807
min,3.573191e-07,2.994513e-07,0.000839,-2.625927
25%,5.265808e-02,4.107684e-02,6.565171,0.566433
50%,1.242156e-01,1.007455e-01,11.249556,0.763968
75%,3.450462e-01,2.680474e-01,17.726814,0.873841
max,2.394120e+00,1.852715e+00,61.237050,1.000000


## 7. Visualización Aleatoria del 10% de los Grupos

Seleccionamos aleatoriamente el 10% de los grupos y graficamos para cada uno: evolución de variables exógenas, resultados de predicción, diferencias porcentuales mes a mes y tendencias.

In [7]:
# Seleccionar 10% de los grupos al azar para visualización
grupos_sample = random.sample(list(grupos), max(1, int(0.01 * len(grupos))))
print(f"Grupos seleccionados para visualización: {grupos_sample}")

# Create df_resultados from resultados_todos
df_resultados = pd.concat(resultados_todos, ignore_index=True) if resultados_todos else pd.DataFrame()

import re

def hex_to_rgba(hex_color, alpha=0.2):
    """Convert hex color string to rgba string with given alpha."""
    hex_color = hex_color.lstrip('#')
    if len(hex_color) == 6:
        r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        return f'rgba({r},{g},{b},{alpha})'
    else:
        # fallback to gray if invalid
        return f'rgba(128,128,128,{alpha})'

for grupo in grupos_sample:
    df_g = df_resultados[df_resultados['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
    if df_g.empty:
        logging.warning(f"No hay datos para el grupo {grupo} en df_resultados")
        continue

    # Evolución de variables exógenas con Plotly
    fig_exog = sp.make_subplots(rows=len(exog_vars), cols=1, 
                                subplot_titles=[f'{grupo} - Evolución de {var}' for var in exog_vars],
                                shared_xaxes=True, vertical_spacing=0.1)

    for i, var in enumerate(exog_vars, start=1):
        hist_g = df_hist[df_hist['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
        fcst1_g = df_fcst1[df_fcst1['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
        fcst2_g = df_fcst2[df_fcst2['REGION_CATEGORIA_PRODUCTO'] == grupo].copy()
        
        # Add traces for each dataset
        fig_exog.add_trace(
            go.Scatter(x=hist_g['MES'], y=hist_g[var], mode='lines', name='Histórico', 
                        line=dict(color='black')),
            row=i, col=1
        )
        fig_exog.add_trace(
            go.Scatter(x=fcst1_g['MES'], y=fcst1_g[var], mode='lines', name='Escenario 1', 
                        line=dict(color='#8B7557')),  
            row=i, col=1
        )
        fig_exog.add_trace(
            go.Scatter(x=fcst2_g['MES'], y=fcst2_g[var], mode='lines', name='Escenario 2', 
                        line=dict(color='#1f77b4')),  
            row=i, col=1
        )

    # Update layout
    fig_exog.update_layout(
        height=300 * len(exog_vars), width=1000,
        title_text=f'Evolución de Variables Exógenas - {grupo}',
        showlegend=True,
                legend_title='Serie',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.3,
            x=0.5,
            xanchor='center')
    )
    fig_exog.update_xaxes(title_text='Fecha', row=len(exog_vars), col=1)
    for i, var in enumerate(exog_vars, start=1):
        fig_exog.update_yaxes(title_text=var, row=i, col=1)
    fig_exog.show()

    # Resultados de predicción con intervalos de confianza
    fig = go.Figure()
    color_list = ['#36332E', '#8B7557', '#57788B']
    for tipo, color in zip(['HISTÓRICO', 'ESCENARIO 1', 'ESCENARIO 2'], color_list):
        df_tipo = df_g[df_g['TIPO'] == tipo]
        # Add main prediction line
        fig.add_trace(go.Scatter(
            x=df_tipo['ds'], 
            y=df_tipo['M3_PRODUCTO'], 
            mode='lines', 
            name=tipo, 
            line=dict(color=color)
        ))
        # Add confidence intervals for forecast scenarios only
        if tipo in ['ESCENARIO 1', 'ESCENARIO 2']:
            # Upper bound
            fig.add_trace(go.Scatter(
                x=df_tipo['ds'], 
                y=df_tipo['IC_95_UPPER'], 
                mode='lines', 
                line=dict(width=0), 
                showlegend=False, 
                hoverinfo='skip'
            ))
            # Lower bound with fill to upper bound
            fig.add_trace(go.Scatter(
                x=df_tipo['ds'], 
                y=df_tipo['IC_95_LOWER'], 
                mode='lines', 
                line=dict(width=0), 
                fill='tonexty', 
                fillcolor=hex_to_rgba(color, 0.2),  # Proper rgba color
                showlegend=False, 
                hoverinfo='skip'
            ))

    fig.update_layout(
        title=f'Predicción de Cantidades Vendidas - {grupo}', 
        xaxis_title='Fecha', 
        yaxis_title='Cantidad Vendida (m3)', 
        legend_title='Serie',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.35,
            x=0.5,
            xanchor='center'),
        width=1000, 
        height=400
    )
    fig.show()

    # Diferencias porcentuales
    fig = go.Figure()
    for tipo, color in zip(['HISTÓRICO', 'ESCENARIO 1', 'ESCENARIO 2'], ['#36332E', '#8B7557', '#57788B']):
        df_tipo = df_g[df_g['TIPO'] == tipo]
        fig.add_trace(go.Bar(x=df_tipo['ds'], y=df_tipo['DIF_PCT'], name=tipo, marker_color=color))
    fig.update_layout(
        title=f'Diferencia porcentual mes a mes - {grupo}', 
        xaxis_title='Fecha', 
        yaxis_title='% Dif.',
        legend_title='Serie',
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=-0.35,
            x=0.5,
            xanchor='center'), 
        width=1000, 
        height=400
    )
    fig.show()

Grupos seleccionados para visualización: ['Atlántico-Bebida de Té-Té Verde 300mL x 6uds', 'Valle del Cauca-Agua-Agua con Gas 500mL x 6uds', 'Atlántico-Gaseosa-Cola 600mL x 24uds', 'Atlántico-Jugo-Jugo Naranja 2000mL x 24uds', 'Antioquia-Agua-Agua con Gas 5L x 1uds', 'Antioquia-Jugo-Jugo Lima 2000mL x 12uds', 'Boyacá-Agua-Agua con Gas 5L x 1uds', 'Santander-Jugo-Jugo Lima 2000mL x 12uds', 'Tolima-Agua-Agua Sin Gas 1L x 1uds', 'Bolívar-Gaseosa-Cola 600mL x 24uds', 'Norte de Santander-Bebida de Té-Té Verde 300mL x 6uds', 'Boyacá-Jugo-Jugo Lima 1000mL x 6uds', 'Valle del Cauca-Bebida de Té-Té Verde 300mL x 6uds', 'Norte de Santander-Agua-Agua Sin Gas 1L x 1uds']


## 6. Consolidación y Exportación de Resultados a Snowflake

Unificamos los resultados y métricas de todos los grupos en dataframes finales y exportamos a Snowflake usando COPY INTO e INSERT INTO.

In [8]:
# Consolidate results
df_resultados = pd.concat(resultados_todos, ignore_index=True) if resultados_todos else pd.DataFrame()

# Add split columns
split_cols = df_resultados['REGION_CATEGORIA_PRODUCTO'].str.split('-', n=2, expand=True)
df_resultados['REGION'] = split_cols[0]
df_resultados['CATEGORIA'] = split_cols[1]
df_resultados['PRODUCTO'] = split_cols[2]
df_resultados = df_resultados.rename(columns={'ds': 'FECHA'})

# Validate row count
n_hist = len(df_hist)
n_fcst1 = len(df_fcst1)
n_fcst2 = len(df_fcst2)
suma_esperada = n_hist + n_fcst1 + n_fcst2
suma_real = len(df_resultados)
print(f"Filas esperadas (histórico + esc1 + esc2): {suma_esperada}")
print(f"Filas reales en df_resultados: {suma_real}")
if suma_esperada == suma_real:
    print("✔️ La suma de filas es correcta.")
else:
    logging.warning(f"❌ Diferencia detectada: esperadas {suma_esperada}, reales {suma_real}")

Filas esperadas (histórico + esc1 + esc2): 7344
Filas reales en df_resultados: 7344
✔️ La suma de filas es correcta.


## 7. Envío de datos a la base de datos y notificación de ejecución a Discord

In [ ]:
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
import pandas as pd

def export_prophet_results_to_snowflake(df_resultados, df_metricas):
    conn = get_snowflake_connection()  
    cursor = conn.cursor()
    try:
      
        cursor.execute("USE SCHEMA BEBIDAS_PROJECT.BEBIDAS_ANALYTICS")

        # Verify UDF exists
        cursor.execute("""
            SELECT * FROM information_schema.functions 
            WHERE function_schema = UPPER('BEBIDAS_ANALYTICS') 
            AND function_name = 'SEND_DISCORD_WEBHOOK';
        """)
        udf_exists = cursor.fetchone()
        if not udf_exists:
            raise Exception("UDF send_discord_webhook does not exist in schema BEBIDAS_PROJECT.BEBIDAS_ANALYTICS. Please create it.")

        df_resultados = df_resultados.copy()
        df_metricas = df_metricas.copy()
        timestamp_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        df_resultados['TIMESTAMP_COL'] = timestamp_str
        df_metricas['TIMESTAMP_COL'] = timestamp_str

        # Convert FECHA column
        if 'FECHA' in df_resultados.columns:
            if pd.api.types.is_numeric_dtype(df_resultados['FECHA']):
                try:
                    df_resultados['FECHA'] = pd.to_datetime(df_resultados['FECHA'], unit='ns').dt.date
                except ValueError:
                    df_resultados['FECHA'] = pd.to_datetime(df_resultados['FECHA'], unit='ms').dt.date
            else:
                df_resultados['FECHA'] = pd.to_datetime(df_resultados['FECHA']).dt.date

        cols_result = ['TIMESTAMP_COL', 'REGION_CATEGORIA_PRODUCTO', 'REGION', 'CATEGORIA', 
                       'PRODUCTO', 'FECHA', 'TIPO', 'M3_PRODUCTO', 'IC_95_LOWER', 
                       'IC_95_UPPER', 'DIF_PCT']
        cols_metrics = ['TIMESTAMP_COL', 'REGION_CATEGORIA_PRODUCTO', 'RMSE', 'MAE', 
                        'SMAPE', 'R2', 'QUALITY']
        df_resultados = df_resultados[cols_result]
        df_metricas = df_metricas[cols_metrics]

        # Create tables
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS PROPHET_M3_RESULTADOS (
                ID INTEGER AUTOINCREMENT,
                TIMESTAMP_COL TIMESTAMP_NTZ,
                REGION_CATEGORIA_PRODUCTO VARCHAR(300),
                REGION VARCHAR(100),
                CATEGORIA VARCHAR(100),
                PRODUCTO VARCHAR(100),
                FECHA DATE,
                TIPO VARCHAR(30),
                M3_PRODUCTO FLOAT,
                IC_95_LOWER FLOAT,
                IC_95_UPPER FLOAT,
                DIF_PCT FLOAT
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS PROPHET_M3_METRICS (
                ID INTEGER AUTOINCREMENT,
                TIMESTAMP_COL TIMESTAMP_NTZ,
                REGION_CATEGORIA_PRODUCTO VARCHAR(300),
                RMSE FLOAT,
                MAE FLOAT,
                SMAPE FLOAT,
                R2 FLOAT,
                QUALITY VARCHAR(20)
            )
        """)

        # Truncate tables
        cursor.execute("TRUNCATE TABLE PROPHET_M3_RESULTADOS")


        # Write DataFrames
        success_resultados, nchunks, nrows, _ = write_pandas(
            conn=conn,
            df=df_resultados,
            table_name='PROPHET_M3_RESULTADOS',
            auto_create_table=False,
            overwrite=False
        )
        print(f"Exported {nrows} rows to PROPHET_M3_RESULTADOS")

        success_metrics, nchunks, nrows, _ = write_pandas(
            conn=conn,
            df=df_metricas,
            table_name='PROPHET_M3_METRICS',
            auto_create_table=False,
            overwrite=False
        )
        print(f"Exported {nrows} rows to PROPHET_M3_METRICS")

       
        cursor.execute('SELECT COUNT(*) FROM PROPHET_M3_RESULTADOS')
        count_resultados = cursor.fetchone()[0]
        cursor.execute('SELECT COUNT(*) FROM PROPHET_M3_METRICS')
        count_metrics = cursor.fetchone()[0]
        enviados_resultados = len(df_resultados)
        enviados_metricas = len(df_metricas)
        mensaje = (
            f"Datos enviados a Snowflake:\n"
            f"- Resultados: {enviados_resultados}\n"
            f"- Métricas: {enviados_metricas}\n\n"
            f"Conteo actual en tablas Snowflake:\n"
            f"- PROPHET_M3_RESULTADOS: {count_resultados}\n"
            f"- PROPHET_M3_METRICS: {count_metrics}"
        )

        # Call UDF
        cursor.execute(
            f"SELECT BEBIDAS_PROJECT.BEBIDAS_ANALYTICS.send_discord_webhook(?, ?)",
            (mensaje, True)
        )
        discord_result = cursor.fetchone()[0]
        print(discord_result)

        conn.commit()
        
    except Exception as e:
        conn.rollback()
        cursor.execute(
            f"SELECT BEBIDAS_PROJECT.BEBIDAS_ANALYTICS.send_discord_webhook(?, ?)",
            (f"Error al exportar o consultar Snowflake: {str(e)}", False)
        )
        discord_result = cursor.fetchone()[0]
        print(discord_result)
    finally:
        cursor.close()
        conn.close()


export_prophet_results_to_snowflake(df_resultados, df_metricas)